In [11]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import (
    mean_squared_error,
    r2_score,
    mean_absolute_error,
    median_absolute_error,
    explained_variance_score
)

# Load the dataset
data = loadmat('./traffic_dataset.mat')  # Adjust the path to your dataset

# Extracting training and testing data
X_train_raw = data['tra_X_tr'][0]  # Training set input data
y_train_raw = data['tra_Y_tr']  # Training set output data
X_test_raw = data['tra_X_te'][0]  # Test set input data
y_test_raw = data['tra_Y_te']  # Test set output data

# Function to convert arrays of matrices into a 2D numpy array (flattening spatial-temporal data)
def flatten_matrices(matrices):
    flattened = np.array([mat.toarray().flatten() if hasattr(mat, 'toarray') else mat.flatten() for mat in matrices])
    return flattened

# Flattening the input matrices
X_train_flattened = flatten_matrices(X_train_raw)
X_test_flattened = flatten_matrices(X_test_raw)

# Averaging the traffic flow across all locations for each time point for both training and testing sets
y_train_aggregated = np.mean(y_train_raw, axis=0)
y_test_aggregated = np.mean(y_test_raw, axis=0)

# Splitting the training data further into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_flattened, y_train_aggregated, test_size=0.2, random_state=42)

# Initializing the Random Forest model
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Training the model with the training data
model.fit(X_train, y_train)
predictions_train = model.predict(X_train)
predictions_val = model.predict(X_val)

# Calculate evaluation metrics for both training and validation sets
rmse_train = np.sqrt(mean_squared_error(y_train, predictions_train))
rmse_val = np.sqrt(mean_squared_error(y_val, predictions_val))
r2_train = r2_score(y_train, predictions_train)
r2_val = r2_score(y_val, predictions_val)

# Output the evaluation metrics
print(f'Training RMSE: {rmse_train}')
print(f'Validation RMSE: {rmse_val}')
print(f'Training R²: {r2_train}')
print(f'Validation R²: {r2_val}')

mae_train = mean_absolute_error(y_train, predictions_train)
mae_val = mean_absolute_error(y_val, predictions_val)
medae_train = median_absolute_error(y_train, predictions_train)
medae_val = median_absolute_error(y_val, predictions_val)
evs_train = explained_variance_score(y_train, predictions_train)
evs_val = explained_variance_score(y_val, predictions_val)

# Output the additional evaluation metrics
print(f'Training MAE: {mae_train}')
print(f'Validation MAE: {mae_val}')
print(f'Training Median AE: {medae_train}')
print(f'Validation Median AE: {medae_val}')
print(f'Training Explained Variance Score: {evs_train}')
print(f'Validation Explained Variance Score: {evs_val}')

# Calculate residuals for the validation set
residuals = y_val - predictions_val

# Plotting Residuals
plt.figure(figsize=(10, 6))
plt.scatter(y_val, residuals)
plt.hlines(y=0, xmin=y_val.min(), xmax=y_val.max(), colors='red', linestyles='--')
plt.xlabel('Predicted')
plt.ylabel('Residuals')
plt.title('Residual Plot')
plt.show()

# Plotting Feature Importances
feature_importances = model.feature_importances_
plt.figure(figsize=(10, 6))
indices = np.argsort(feature_importances)[::-1]
plt.title('Feature Importances')
plt.bar(range(X_train.shape[1]), feature_importances[indices],
        color="r", align="center")
plt.xticks(range(X_train.shape[1]), indices)
plt.xlim([-1, X_train.shape[1]])
plt.xlabel('Feature Index')
plt.ylabel('Importance')
plt.show()

# Plotting Prediction Error Histogram
plt.figure(figsize=(10, 6))
sns.histplot(residuals, bins=50, kde=True)
plt.xlabel('Prediction Error')
plt.ylabel('Frequency')
plt.title('Prediction Error Histogram')
plt.show()

# Plotting the Learning Curves
train_errors, val_errors = [], []
step_size = len(X_train) // 10  # Incremental step size for training
for m in range(1, len(X_train), step_size):
    model = RandomForestRegressor(n_estimators=10, max_depth=5, random_state=42)  # More simplistic model
    model.fit(X_train[:m], y_train[:m])
    y_train_predict = model.predict(X_train[:m])
    y_val_predict = model.predict(X_val)
    train_errors.append(mean_squared_error(y_train[:m], y_train_predict))
    val_errors.append(mean_squared_error(y_val, y_val_predict))

plt.figure(figsize=(10, 6))
plt.plot(np.sqrt(train_errors), label='Train')
plt.plot(np.sqrt(val_errors), label='Validation')
plt.xlabel('Training set size')
plt.ylabel('RMSE')
plt.title('Learning Curves')
plt.legend()
plt.show()

Training RMSE: 0.0059374035604384595
Validation RMSE: 0.015134064068337937
Training R²: 0.9982846326177228
Validation R²: 0.9888912871326031
Training MAE: 0.0040333351919400115
Validation MAE: 0.010508312005377567
Training Median AE: 0.002700710986558566
Validation Median AE: 0.007196663033888606
Training Explained Variance Score: 0.9982850147788955
Validation Explained Variance Score: 0.9889121441724594


KeyboardInterrupt: 